In [2]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

#Get list of teams
week1 = pd.read_csv('Week1.csv')
teams = set(week1['Team'])

#Create empty table
table = DataFrame(columns=[['H','H','H','H','H','A','A','A','A','A'],
                         ['W','L','D','F','CS','W','L','D','F','CS']],
                  index=teams)
table = table.fillna(0)

#Get list of fixtures
fixtures = pd.read_csv('Fixtures.csv')
fixtures = fixtures.set_index('Week')

#For each week so far
for w in range(1,21):
    
    #Load those stats and sum goals and own goals by team into a new dataframe
    stats = pd.read_csv('week%s.csv'%w)
    goals = stats[['Team','Player','G','GON']]
    goal_totals = goals.groupby(['Team']).sum()

    #Get that week's home and away teams
    home = list(fixtures.ix[w,'Home'])
    away = list(fixtures.ix[w,'Away'])

    #For each fixture
    for x in range(0,len(home)):
        
        #Add own goals into the goals total for home and away teams
        hg = goal_totals['G'][home[x]] + goal_totals['GON'][away[x]]
        ag = goal_totals['G'][away[x]] + goal_totals['GON'][home[x]]
        
        #Update the goals for and against
        table['H','F'][home[x]] += hg
        if (hg == 0) : table['A','CS'][away[x]] += 1
        table['A','F'][away[x]] += ag
        if (ag == 0) : table['H','CS'][home[x]] += 1
        
        #Update WLD records
        if hg>ag :
            table['H','W'][home[x]] += 1
            table['A','L'][away[x]] += 1
        elif hg==ag :
            table['H','D'][home[x]] += 1
            table['A','D'][away[x]] += 1
        else :
            table['H','L'][home[x]] += 1
            table['A','W'][away[x]] += 1
            
#Change G to GpG and calculate points
table['H','F'] = table['H','F'] / (table['H','W'] + table['H','D'] + table['H','L'])
table['A','F'] = table['A','F'] / (table['A','W'] + table['A','D'] + table['A','L'])
table['T','Pts'] = 3*(table['H','W'] + table['A','W']) + table['H','D'] + table['A','D']

#Sort according to points
table.sort_values(('T','Pts'), ascending=False, inplace=True)
    
print table

    

      H                     A                      T
      W  L  D         F CS  W  L  D         F CS Pts
ARS   7  1  2  1.600000  5  6  3  1  1.800000  4  42
LEIC  5  1  4  1.600000  3  6  1  3  2.100000  2  40
MCI   8  2  0  2.900000  2  4  3  3  1.000000  6  39
TOT   5  1  4  1.800000  5  4  1  5  1.600000  2  36
MUN   5  1  4  1.200000  7  4  4  2  1.200000  3  33
WHU   4  2  4  1.600000  3  4  2  4  1.400000  4  32
CRY   4  5  2  1.090909  4  5  2  2  1.222222  2  31
LIV   4  2  3  1.222222  3  4  4  3  1.000000  4  30
WAT   4  5  2  1.000000  6  4  2  3  1.555556  2  29
STK   4  4  1  1.111111  3  4  3  4  1.000000  5  29
EVE   3  4  4  2.090909  1  3  1  5  1.444444  4  27
WBA   4  5  2  1.272727  3  3  3  3  0.666667  4  26
SOU   4  4  2  1.700000  3  2  4  4  0.900000  3  24
NOR   4  3  3  1.200000  3  2  6  2  1.100000  0  23
CHE   4  4  2  1.500000  3  2  5  3  1.100000  3  23
BOUR  2  3  4  1.111111  2  3  6  2  1.090909  2  21
SWA   3  3  4  0.900000  4  1  6  3  0.800000 